## Installing PyTorch and Pillow == 0.4

In [13]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5c27a000 @  0x7eff28e6c2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
0.4.0
True


In [14]:
!pip install Pillow==4.0.0 

    100% |████████████████████████████████| 5.6MB 6.4MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


## Imports
All the imports are defined here


In [0]:
%matplotlib inline
import os
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
!ls '/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset'

final_backup_saved_model.pt  modified_data_loader_saved_model.pt  testing
forged			     saved_model.pt			  training


In [0]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(1, 128, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(64, 32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),

            nn.ReflectionPad2d(1),
            nn.Conv2d(32, 16, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(16),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(16, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),


        )

        self.fc1 = nn.Sequential(
            nn.Linear(8*100*100, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 5))

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [0]:
net = SiameseNetwork().cuda()

In [0]:
# torch.save(net,"/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/saved_model.pt")
# net = torch.load("/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/saved_model.pt")

# filepath = "/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/final_backup_saved_model.pt" # 0.76 threshold

filepath = "/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/modified_data_loader_saved_model.pt"

# torch.save(net.state_dict(), filepath)

#Later to restore:
net.load_state_dict(torch.load(filepath))

In [21]:
!ls '/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/'

final_backup_saved_model.pt  modified_data_loader_saved_model.pt  testing
forged			     saved_model.pt			  training


In [0]:
def image_loader(loader, image_name):
    image = Image.open(image_name)
    image = image.convert("L")
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image

data_transforms = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])

In [0]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [0]:
col_names =  ['filename', 'confidence_score', 'decision']
submision_csv  = pd.DataFrame(columns = col_names)

In [0]:
names = [os.path.basename(x) for x in glob.glob('/content/drive/My Drive/Colab Notebooks/axis_data/final_sub/final_submission_test/*.jpg')]
submision_csv['filename'] = names

In [0]:
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/axis_data/final_sub/final_submission_test/'
USER_REG_DIR = '/content/drive/My Drive/Colab Notebooks/axis_data/final_sub/final_submission_ordered/'

for row in submision_csv.itertuples():
    file = row.filename
    file_name_initial = re.search(r'^.*(?=(\_))',file).group(0)
    user_id = file_name_initial[3:]
    confidence = 0
    user_signs = glob.glob(USER_REG_DIR+user_id+'/*.jpg')
    test_image = TEST_DIR + file
    test_image = image_loader(data_transforms, test_image)
    
    for genuine_sign in user_signs:
      user_sign = image_loader(data_transforms, genuine_sign)
      output1,output2 = net(Variable(user_sign).cuda(),Variable(test_image).cuda())
      euclidean_distance = F.pairwise_distance(output1, output2)
      euclidean_distance = euclidean_distance.cpu().detach().numpy()
      confidence = confidence + euclidean_distance
    
    confidence = confidence/3
    submision_csv.loc[row.Index, 'confidence_score'] = confidence[0]

In [0]:
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/axis_data/final_sub/final_submission_test/'
# ^.*(?=(\.txt))
file = 'U11001_380.jpg'
file_name = re.search(r'^.*(?=(\_))',file).group(0)
file_name


In [28]:
submision_csv.head()

,filename,confidence_score,decision
0,U11001_125.jpg,0.463007,NaN
1,U11001_124.jpg,0.55448,NaN
2,U11001_273.jpg,0.478525,NaN
3,U11001_127.jpg,0.753847,NaN
4,U11001_343.jpg,0.564291,NaN


In [0]:
for row in submision_csv.itertuples():
  submision_csv.loc[row.Index, 'decision'] = 0
  score = row.confidence_score
  if score > 0.6:
    submision_csv.loc[row.Index, 'decision'] = 1

In [30]:
submision_csv.head()

,filename,confidence_score,decision
0,U11001_125.jpg,0.463007,0
1,U11001_124.jpg,0.55448,0
2,U11001_273.jpg,0.478525,0
3,U11001_127.jpg,0.753847,1
4,U11001_343.jpg,0.564291,0


In [0]:
# submision_csv.to_csv('/content/drive/My Drive/Colab Notebooks/axis_data/confidence_metric.csv')  
submision_csv.to_csv('/content/drive/My Drive/Colab Notebooks/axis_data/final_sub_corrected.csv')